In [4]:
# %pip install -U ollama openai

# 

# Simple Tool Calling for LLM + GMAIL

In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
from datetime import datetime

# Email credentials and SMTP settings
smtp_server = "smtp.gmail.com"
port = 587  # For starttls
sender_email = "lucasenkrateia@gmail.com"  # Enter your email
password = os.environ["GMAIL_PWD"]

def send_email(to, email_contents):
    """
    Send an email using the configured sender email and SMTP settings
    
    Args:
        to (str): Email address of the recipient
        subject (str): Subject line of the email
        body (str): Body text of the email
    """
    # Create message
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = to
    message["Subject"] = "Test email"
    message.attach(MIMEText(email_contents, "plain"))

    # Send email
    try:
        server = smtplib.SMTP(smtp_server, port)
        server.starttls()  # Secure the connection
        server.login(sender_email, password)
        server.sendmail(sender_email, to, message.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        server.quit()

# Example usage
today_date = datetime.now().strftime('%d %B %Y')
email_subject = f"Test Email Lucas LOcal LLMs"
email_body = f"Hi Lucas! Today's date is {today_date}."
# send_email("lucasbnsoares@hotmail.com", email_subject, email_body)

![](./assets-resources/2024-12-02-18-14-58.png)


In [6]:
import ollama


response = ollama.chat(
    model="llama3.2",  # llama3.2 has reliable tool calling support
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant \
            that helps users craft and send their emails using the send_email tool.'},
        {'role': 'user', 'content': 'Can you send an email \
            to lucasbnsoares@hotmail.com, containing a nice motivational message?"'},
    ],
    tools=[{
        'type': 'function',
        'function': {
            'name': 'send_email',
            'description': 'Send an email to a user, the input params are to and email_contents.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'to': {
                        'type': 'string',
                        'description': 'The email address of the receiver',
                    },
                    'email_contents': {
                        'type': 'string',
                        'description': 'The body of the email',
                    }
                },
                'required': ['to', 'email_contents']
            }
        }
    }],
)

print(response)

model='llama3.2' created_at='2026-02-03T19:18:41.245616Z' done=True done_reason='stop' total_duration=858763084 load_duration=82833125 prompt_eval_count=218 prompt_eval_duration=79407125 eval_count=50 eval_duration=443662584 message=Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=[ToolCall(function=Function(name='send_email', arguments={'to': 'lucasbnsoares@hotmail.com', 'email_contents': '{"type":"string","description":"The body of the email","email_contents":"A nice motivational message"}'}))]) logprobs=None


In [7]:
response

ChatResponse(model='llama3.2', created_at='2026-02-03T19:18:41.245616Z', done=True, done_reason='stop', total_duration=858763084, load_duration=82833125, prompt_eval_count=218, prompt_eval_duration=79407125, eval_count=50, eval_duration=443662584, message=Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=[ToolCall(function=Function(name='send_email', arguments={'to': 'lucasbnsoares@hotmail.com', 'email_contents': '{"type":"string","description":"The body of the email","email_contents":"A nice motivational message"}'}))]), logprobs=None)

In [8]:
response['message']

Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=[ToolCall(function=Function(name='send_email', arguments={'to': 'lucasbnsoares@hotmail.com', 'email_contents': '{"type":"string","description":"The body of the email","email_contents":"A nice motivational message"}'}))])

In [9]:
def run_tool_calls(response):
    tool_calls = response['message']['tool_calls']
    if not tool_calls:
        print("No tool calls found in response")
        return
    for tool_call in tool_calls:
        function_name = tool_call['function']['name']
        arguments = tool_call['function']['arguments']
        if function_name == 'send_email':
            to = arguments['to']
            email_contents = arguments['email_contents']
            send_email(to, email_contents)
            print(f"Email sent to {to} with body: {email_contents}")
        else:
            print(f"Unknown tool call: {function_name}")

run_tool_calls(response)

Email sent successfully!
Email sent to lucasbnsoares@hotmail.com with body: {"type":"string","description":"The body of the email","email_contents":"A nice motivational message"}


![](2025-10-03-17-46-25.png)

# Using OpenAI API + Ollama

In [ ]:
from openai import OpenAI 

client = OpenAI()

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "Can you send an email to lucasbnsoares@hotmail.com, containing 3 things you think are essential for someone to be a life long learner?"}
    ],
)

response

ChatCompletion(id='chatcmpl-CMdF22AUQ91645Fbb3zLsPYjAeU0j', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I don't have the capability to send emails directly. However, I can help you draft an email that you can send through your email service. Here’s a draft:\n\n---\n\nSubject: Essential Traits for Lifelong Learning\n\nHi Lucas,\n\nI hope this message finds you well. I wanted to share a few thoughts I've been pondering about lifelong learning. Here are three essential traits that I believe are crucial for anyone looking to embrace this journey:\n\n1. **Curiosity**: Cultivating a sense of curiosity helps you remain open to new experiences and knowledge. It drives you to ask questions, explore new fields, and seek out diverse perspectives.\n\n2. **Adaptability**: In a constantly changing world, being adaptable allows you to learn from both successes and failures. It helps you adjust to new situations, technologies, and ideas, ensurin

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that helps users craft and send their emails using the send_email tool."},
    {"role": "user", "content": "Can you send an email to lucasbnsoares@hotmail.com, containing 3 things you think are essential for someone to be a life long learner?"}
]

In [ ]:
tools=[{
        'type': 'function',
        'function': {
            'name': 'send_email',
            'description': 'Send an email to a user, the input params are to, subject and body.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'to': {
                        'type': 'string',
                        'description': 'The email address of the receiver',
                    },
                    'subject': {
                        'type': 'string',
                        'description': 'The subject of the email',
                    },
                    'body': {
                        'type': 'string',
                        'description': 'The body of the email',
                    }
                },
                'required': ['to', 'subject', 'body']
            }
        }
    }]



response = client.chat.completions.create(model="llama3.2", messages=messages, tools=tools)

print(response)

In [ ]:
response.choices[0].message.tool_calls